In [65]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os, sys

PATH = "../Data/Images/"

img1 = cv.imread(f'{PATH}G1.JPG')
img2 = cv.imread(f'{PATH}G14.JPG')
gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

In [66]:
def Show(img):
    cv.namedWindow('raw', cv.WINDOW_KEEPRATIO)
    cv.imshow('raw', img)
    cv.resizeWindow('raw', 1000, 1000)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
def FindKeyLocationPosition(image_2darray, currKp_2darray, xMin, xMax, yMin, yMax):
    currKp_2darray[xMin:xMax, yMin:yMax] = 1
    return currKp_2darray

def VisualizeDetector(img1, img2):

    cv.namedWindow('ORB', cv.WINDOW_NORMAL)

    def f(x):
        return

    # Initiate ORB detector
    cv.createTrackbar('Edge Threshold', 'ORB', 100, 1000, f)
    cv.createTrackbar('Patch Size', 'ORB', 100, 200, f)
    cv.createTrackbar('N Levels', 'ORB', 10, 30, f)
    cv.createTrackbar('Fast Threshold', 'ORB', 15, 50, f)
    cv.createTrackbar('Scale Factor', 'ORB', 12, 25, f)
    cv.createTrackbar('WTA K', 'ORB', 2, 4, f)
    cv.createTrackbar('First Level', 'ORB', 0, 20, f)
    cv.createTrackbar('N Features', 'ORB', 250, 500, f)

    while True:
        edge_threshold = cv.getTrackbarPos('Edge Threshold', 'ORB')
        patch_size = cv.getTrackbarPos('Patch Size', 'ORB')
        n_levels = cv.getTrackbarPos('N Levels', 'ORB')
        fast_threshold = cv.getTrackbarPos('Fast Threshold', 'ORB')
        scale_factor = cv.getTrackbarPos('Scale Factor', 'ORB') / 10
        wta_k = cv.getTrackbarPos('WTA K', 'ORB')
        first_level = cv.getTrackbarPos('First Level', 'ORB')
        n_features = cv.getTrackbarPos('N Features', 'ORB')

        if wta_k < 2:
            wta_k = 2

        if patch_size < 2:
            patch_size = 2

        if n_levels < 1:
            n_levels = 1

        if scale_factor < 1:
            scale_factor = 1

        orb = cv.ORB_create(edgeThreshold=edge_threshold, patchSize=patch_size, nlevels=n_levels, fastThreshold=fast_threshold, scaleFactor=scale_factor, WTA_K=wta_k,scoreType=cv.ORB_HARRIS_SCORE, firstLevel=first_level, nfeatures=n_features)

        #Orb parameters patch1 100, 100, 6, 15, 12, 2, 0, 50
        #Orb parameters fullImg 100, 100, 10, 15, 12, 2, 0, 250

        # find the keypoints with ORB
        kp1 = orb.detect(img1,None)
        kp2 = orb.detect(img2,None)
        # compute the descriptors with ORB
        kp1, des1 = orb.compute(img1, kp1)
        kp2, des2 = orb.compute(img2, kp2)

        # draw only keypoints location,not size and orientation
        img1B = cv.drawKeypoints(img1, kp1, None, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS).copy()
        img2B = cv.drawKeypoints(img2, kp2, None, color=(0,255,0), flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS).copy()

        finalImg = np.concatenate((img1B, img2B), axis=1)

        cv.imshow('ORB', finalImg)

        if cv.waitKey(10) & 0xFF == 27:
            break

    cv.destroyAllWindows()
    
def ORB(img1, img2, n_features, mask=None):
    orb = cv.ORB_create(edgeThreshold=100, patchSize=100, nlevels=10, fastThreshold=15, scaleFactor=1.2, WTA_K=2,scoreType=cv.ORB_HARRIS_SCORE, firstLevel=0, nfeatures=n_features)
    kp1, des1 = orb.detectAndCompute(img1, mask)
    kp2, des2 = orb.detectAndCompute(img2, mask)
    
    return kp1, kp2, des1, des2
    
def Match(img1, img2, kp1, kp2, des1, des2):
    # BFMatcher with default params
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
            
    imgMatch = cv.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    show(imgMatch)
            
    MIN_MATCH_COUNT = 10

    good = np.squeeze(good)
    
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)

        h,w = img1.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        img2T = img2.copy()
        img2T = cv.warpPerspective(img2T, M, img2T.T.shape, flags=cv.INTER_LINEAR)

    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
        matchesMask = None

    finalImg = np.concatenate((img1, img2, img2T), axis=1)

    show(finalImg)

In [67]:
#Wall pattern cut

initialArea = np.zeros(img1.shape, dtype=np.uint8)[:,:,0]
cut1 = findKeyLocationPosition(gray1, initialArea, 520, 1036, 633, 1051)*255

kp1, kp2, des1, des2 = ORB(gray1, gray2,150)
# Match(gray1, gray2, kp1, kp2, des1, des2)

kp1Cut, kp2Cut, des1Cut, des2Cut = ORB(gray1, gray2, 50, cut1)
# Match(gray1, gray2, kp1Cut, kp2Cut, des1Cut, des2Cut)

kp1Final = np.hstack((kp1, kp1Cut))
kp2Final = np.hstack((kp2, kp2Cut))
des1Final = np.vstack((des1, des1Cut))
des2Final = np.vstack((des2, des2Cut))

Match(gray1, gray2, kp1Final, kp2Final,des1Final, des2Final)